 # Agente LangChain - Agente de Análise de Cliente para Marketing Telco

 Este notebook implementa um agente que analisa informações de clientes para gerar ações de marketing.

Este agente utiliza sinais gerados por modelos de IA tradicional (churn e valor mensal) para gerar insights de negócio para Marketing.

Ele não toma decisões automáticas.
Ele explica contextos e sugere caminhos.

 ## Instalando bibliotecas

In [18]:
!pip install langchain-oci langchain scikit-learn


## Ferramenta que busca informações do cliente

In [67]:
import pandas as pd
from langchain_core.tools import tool
df = pd.read_csv("telco_churn.csv")

@tool
def buscar_cliente(id_cliente: int) -> dict:
    """
    Retorna os dados de um cliente Telco a partir do id_cliente.
    """
    cliente = df[df["id_cliente"] == id_cliente]
    cliente = cliente.drop(columns=["churn"])
    
    if cliente.empty:
        return "Cliente não está na base"
    
    return cliente.to_dict(orient="records")[0]


## Ferramenta que calcula a probabilidade de churn do cliente

In [68]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

X = df.drop(columns=['id_cliente', 'churn'])
y = df['churn']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model = LogisticRegression()
model.fit(X_train_scaled, y_train)

y_pred = model.predict(X_test_scaled)

@tool
def prever_churn_cliente(id_cliente: int) -> str:
    """
    Retorna a probabilidade de churn de um cliente específico.
    """
    # Buscar cliente
    cliente = df[df["id_cliente"] == id_cliente]

    if cliente.empty:
        return json.dumps({
            "erro": "Cliente não encontrado"
        })

    # Preparar features (mesma ordem do treino!)
    X_cliente = cliente[X.columns]

    # Escalar
    X_cliente_scaled = scaler.transform(X_cliente)

    # Predizer probabilidade
    churn_prob = model.predict_proba(X_cliente_scaled)[0][1]

    # Retorno estruturado (ideal para agente)
    return json.dumps({
        "id_cliente": int(id_cliente),
        "probabilidade_churn": round(float(churn_prob), 2),
        "interpretacao": (
            "Risco alto de churn" if churn_prob > 0.7
            else "Risco médio de churn" if churn_prob > 0.4
            else "Risco baixo de churn"
        )
    })


 ## Configuração de autenticação de OCI SDK
    - https://github.com/oracle/oci-python-sdk/blob/master/examples/configuration_example.py

 ## Importando bibliotecas e callback handler
 Callback é uma configuração para conseguir visualizar passos internos do caminho do agente

In [69]:
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
from langchain_oci.chat_models import ChatOCIGenAI
from langchain.agents import create_agent
from langchain_core.prompts import PromptTemplate
from langchain_core.callbacks import BaseCallbackHandler
from langchain_core.messages import AIMessage
import json



class CleanAgentCallback(BaseCallbackHandler):
    def on_tool_start(self, serialized, input_str, **kwargs):
        tool_name = serialized.get("name", "unknown_tool")
        print(f"\n TOOL: {tool_name}")
        print(f"   ↳ input: {input_str}")

    def on_tool_end(self, output, **kwargs):
        print("TOOL RESULT:")

        content = getattr(output, "content", output)

        try:
            data = json.loads(content)
            print(json.dumps(data, indent=2, ensure_ascii=False))
        except Exception:
            print(content)



## Configuração do modelo LLM do agente (Cerebro)
Observe que a região deve ser a mesma que a da sua chave, se precisar, altere abaixo.

#### Cohere 
- "cohere.command-a-03-2025", (sp)
- "cohere.command-latest", (sp)
- "cohere.command-plus-latest", (sp)
- "cohere.command-r-08-2024", (sp)
- "cohere.command-r-plus-08-2024", (sp)

#### Google Gemini (apenas us)
- "google.gemini-2.5-pro", 
- "google.gemini-2.5-flash",
- "google.gemini-2.5-flash-lite",

#### Meta Llama
- "meta.llama-3.1-405b-instruct", (sp)
- "meta.llama-3.2-90b-vision-instruct", (sp)
- "meta.llama-3.3-70b-instruct", (sp)
- "meta.llama-4-maverick-17b-128e-instruct-fp8",
- "meta.llama-4-scout-17b-16e-instruct",

#### OpenAI OSS (apenas us)
- "openai.gpt-oss-120b",
- "openai.gpt-oss-20b",

#### xAI Grok (apenas us)
- "xai.grok-code-fast-1",
- "xai.grok-4-fast-non-reasoning",
- "xai.grok-4-fast-reasoning",
- "xai.grok-4",
- "xai.grok-3",
- "xai.grok-3-mini",
- "xai.grok-3-mini-fast",
- "xai.grok-3-fast"

In [70]:
region = "us-chicago-1"
model_name = "xai.grok-3-mini-fast"

In [71]:
llm = ChatOCIGenAI(
  model_id=model_name,
  service_endpoint=f"https://inference.generativeai.{region}.oci.oraclecloud.com",
  compartment_id="ocid1.compartment.oc1..aaaaaaaa3x7n7wwfnghe4imvt3niwo76wgqv6ecn2iadiwoph73jjowbhbna",
  provider="meta",
  model_kwargs={
    "temperature": 0.3, 
    "max_tokens": 800,   
    "top_p": 0.8, 
  },
  auth_type="API_KEY",
  auth_profile="DEFAULT",
  auth_file_location = "./config"
)

## Adicionando ferramentas

In [72]:
tools = [
    buscar_cliente,
    prever_churn_cliente
]

## Prompt do agente e criação do agente

In [81]:
prompt = """
Você é um analista sênior de Marketing na Claro.

Você deve, quando necessário:
- Consultar os dados do cliente
- Consultar a probabilidade de churn via ferramenta apropriada
- Analisar o contexto do cliente (uso, valor, histórico)
- Construir uma proposta de marketing coerente e explicável
- Caso a proposta seja realizar uma medida de retenção, gerar uma proposta de contato para o cliente ao final.

IMPORTANTE:
- NÃO use regras fixas
- NÃO invente informações! Use apenas os dados captados nas ferramentas para fazer sua analise.
- NÃO retorne apenas rótulos como "alto" ou "baixo"
- Sempre explique o raciocínio por trás da proposta

Você tem acesso às seguintes ferramentas:

{tools}

Use EXATAMENTE o formato abaixo:

Question: a pergunta recebida
Thought: o que você precisa analisar e quais informações buscar
Action: a ferramenta que você irá usar
Action Input: o input da ferramenta
Observation: o resultado retornado pela ferramenta
Thought: análise integrada dos dados e sinais obtidos
Final Answer: 
- Um resumo do perfil do cliente
- A probabilidade de churn (em percentual)
- Uma proposta de ação de marketing contextualizada
- A justificativa da proposta, em linguagem de negócio

Sempre responda em português do Brasil.

Begin!
"""


agent = create_agent(llm, tools, system_prompt=prompt)

## Processar resposta final para deixar mais bonito (opcional)

In [82]:
def extract_final_output(agent_response):
    final_text = None
    total_tokens = None

    for msg in reversed(agent_response["messages"]):
        if isinstance(msg, AIMessage):
            if msg.content and final_text is None:
                if "Final Answer:" in msg.content:
                    final_text = msg.content.split("Final Answer:", 1)[1].strip()
                else:
                    final_text = msg.content.strip()

            if total_tokens is None:
                total_tokens = (
                    msg.response_metadata.get("total_tokens")
                    or msg.additional_kwargs.get("total_tokens")
                )

        if final_text and total_tokens:
            break

    return final_text, total_tokens

## Execução prática do agente

In [83]:
pergunta = """
Analise o cliente de id 1024.
Explique o comportamento dele e aponte possíveis riscos ou oportunidades
para ações de marketing.
"""

callback = CleanAgentCallback()

response = agent.invoke(
    {"messages": pergunta},
    config={"callbacks": [callback]}
)

final_answer, total_tokens = extract_final_output(response)

print("\nFinal answer:")
print(final_answer)

print("\nMetadados:")
print(f"total tokens: {total_tokens}")


 TOOL: buscar_cliente
   ↳ input: {'id_cliente': 1024}
TOOL RESULT:
{
  "id_cliente": 1024,
  "tempo_contrato_meses": 3,
  "valor_mensal": 185,
  "uso_dados_gb": 5,
  "qtd_reclamacoes": 2,
  "atraso_pagamento": 1,
  "recebeu_campanha": 0
}

 TOOL: prever_churn_cliente
   ↳ input: {'id_cliente': 1024}
TOOL RESULT:
{
  "id_cliente": 1024,
  "probabilidade_churn": 0.97,
  "interpretacao": "Risco alto de churn"
}

Final answer:
- **Resumo do perfil do cliente:** O cliente com ID 1024 tem um contrato de apenas 3 meses, indicando um relacionamento inicial e potencialmente frágil com a Claro. Ele paga R$ 185 mensalmente, mas utiliza apenas 5 GB de dados, o que sugere subutilização do plano e pouca interação com os serviços oferecidos. Além disso, há 2 reclamações registradas e 1 atraso no pagamento, o que pode refletir insatisfação ou desafios financeiros, e ele nunca recebeu campanhas de marketing, o que representa uma lacuna em nossos esforços de engajamento.
- **Probabilidade de churn:** 

## Segundo cenário

In [86]:
pergunta = f"cliente 1010"
response = agent.invoke(
    {"messages": pergunta},
    config={"callbacks": [callback]}
)

final_answer, total_tokens = extract_final_output(response)

print("\nFinal answer:")
print(final_answer)

print("\nMetadados:")
print(f"total tokens: {total_tokens}")


 TOOL: buscar_cliente
   ↳ input: {'id_cliente': 1010}
TOOL RESULT:
{
  "id_cliente": 1010,
  "tempo_contrato_meses": 60,
  "valor_mensal": 90,
  "uso_dados_gb": 35,
  "qtd_reclamacoes": 0,
  "atraso_pagamento": 0,
  "recebeu_campanha": 1
}

 TOOL: prever_churn_cliente
   ↳ input: {'id_cliente': 1010}
TOOL RESULT:
{
  "id_cliente": 1010,
  "probabilidade_churn": 0.01,
  "interpretacao": "Risco baixo de churn"
}

Final answer:
- **Resumo do perfil do cliente:** O cliente 1010 é um assinante leal da Claro, com 60 meses de contrato, o que indica uma relação de longo prazo. Ele gasta R$90 mensalmente e utiliza cerca de 35 GB de dados por mês, sem nenhuma reclamação registrada ou atrasos em pagamentos. Além disso, já recebeu uma campanha promocional anteriormente, o que sugere engajamento prévio com ofertas da empresa.  
- **A probabilidade de churn:** 1% (baseada na previsão obtida, que indica um risco baixo).  
- **Proposta de ação de marketing contextualizada:** Recomendo uma ação de up

In [85]:
pergunta = f"como eu faço um bolo de chocolate?"
response = agent.invoke(
    {"messages": pergunta},
    config={"callbacks": [callback]}
)

final_answer, total_tokens = extract_final_output(response)

print("\nFinal answer:")
print(final_answer)

print("\nMetadados:")
print(f"total tokens: {total_tokens}")


Final answer:
Desculpe, mas eu sou um analista sênior de Marketing na Claro, especializado em assuntos relacionados a serviços de telecomunicações, análise de clientes e estratégias de retenção. Sua pergunta sobre como fazer um bolo de chocolate não está dentro do escopo do meu papel.

Se você tiver dúvidas sobre planos Claro, análise de uso de dados, ou qualquer coisa relacionada aos nossos serviços, é só perguntar! Caso contrário, posso te recomendar buscar essa informação em sites de receitas ou vídeos tutoriais. Como posso ajudar com algo relacionado à Claro?

Metadados:
total tokens: 1061
